In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv('C:/Intern/HnM/HM_all_stores.csv')

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['name', 'phone',
       'countryCode', 'timeZoneIndex','streetName1',
       'streetName2', 'address_string', 'state'])

In [ ]:
print("Total duplicate: ", df.duplicated().sum())

In [ ]:
df = df.drop_duplicates()
print("Total duplicate: ", df.duplicated().sum())

In [ ]:
df.isna().sum().sort_values(ascending = False)

In [ ]:
df[['Sun_open_hours', 'Mon_open_hours', 'Tue_open_hours', 'Wed_open_hours', 
    'Thu_open_hours', 'Fri_open_hours', 'Sat_open_hours']] = df[['Sun_open_hours', 'Mon_open_hours', 'Tue_open_hours', 'Wed_open_hours', 
    'Thu_open_hours', 'Fri_open_hours', 'Sat_open_hours']].fillna(0)

df['storeClass'].fillna('Unknown', inplace=True)

In [ ]:
df.sample(5)

In [ ]:
df['Sunday'] = df['Sun_open_hours'].apply(lambda x: 'Closed' if x==0 else 'Open')

In [ ]:
columns = ['Sun_open_hours', 'Mon_open_hours', 'Tue_open_hours', 'Wed_open_hours', 
           'Thu_open_hours', 'Fri_open_hours', 'Sat_open_hours']

def split_and_convert(time_str):
    if time_str == 'Closed' or time_str == '0':
        return pd.NA, pd.NA  
    try:
        # Split the time range into opening and closing times
        open_time, close_time = time_str.split('-')
        # Convert to datetime objects with a fixed date
        open_time = datetime.strptime(open_time, '%H:%M').time()
        close_time = datetime.strptime(close_time, '%H:%M').time()
        return open_time, close_time
    except (ValueError, AttributeError):
        # Return missing values if conversion fails
        return pd.NA, pd.NA

for col in columns:
    df[[f'{col}_open', f'{col}_close']] = df[col].apply(split_and_convert).apply(pd.Series)


def calculate_duration(open_time, close_time):
    if pd.isna(open_time) or pd.isna(close_time):
        return 0
    
    today = datetime.today().date()
    open_datetime = datetime.combine(today, open_time)
    close_datetime = datetime.combine(today, close_time)
    
    if close_datetime < open_datetime:
        close_datetime += timedelta(days=1)
    duration = close_datetime - open_datetime
    return duration.seconds / 3600


for col in columns:
    df[f'{col}_duration'] = df.apply(
        lambda row: calculate_duration(row[f'{col}_open'], row[f'{col}_close']),
        axis=1
    )


df.drop(columns=[col for col in columns for col in [f'{col}_open', f'{col}_close']], inplace=True)


In [ ]:
df.sample(5)

In [ ]:
df['mean_duration_per_week'] = df[['Sun_open_hours_duration', 'Mon_open_hours_duration', 'Tue_open_hours_duration', 
                          'Wed_open_hours_duration', 'Thu_open_hours_duration', 'Fri_open_hours_duration', 
                          'Sat_open_hours_duration']].mean(axis=1)

df = df.drop(columns=['Sun_open_hours_duration', 'Mon_open_hours_duration', 'Tue_open_hours_duration', 
                      'Wed_open_hours_duration', 'Thu_open_hours_duration', 'Fri_open_hours_duration', 
                      'Sat_open_hours_duration'])

In [ ]:
df["mean_duration_per_week"] = df["mean_duration_per_week"].astype(int)

In [ ]:
df.info()

In [ ]:
df.to_csv('HnM_Clean.csv')